主要包含：
* lightning中的数据容器
* 迭代多个数据集
* 处理顺序数据

# Lightning 中的数据容器

Lightning 中使用了几种不同的数据容器：

|  对象   | 定义  |
|  ----  | ----  |
| Dataset  | PyTorchDataset表示从键到数据样本的映射。 |
| IterableDataset  | PyTorch IterableDataset 表示数据流。 |
|DataLoader|PyTorch DataLoader 表示可在 DataSet 上迭代的 Python。|
|LightningDataModule|LightningDataModule 只是一个集合：训练 DataLoader、验证 DataLoader、测试 DataLoader 和预测 DataLoader，以及匹配的转换和所需的数据处理/下载步骤。|

# 为什么选择 LightningDataModules？

LightningDataModule 被设计为一种将数据相关挂钩与 LightningModule 分离的方式，以便您可以开发数据集不可知模型。 LightningDataModule 可以轻松地将不同的数据集与您的模型进行热交换，因此您可以对其进行测试并跨域对其进行基准测试。 它还使跨项目共享和重用确切的数据拆分和转换成为可能。

阅读本文以了解有关 LightningDataModules 的更多详细信息。

# 多个数据集

有几种方法可以将多个数据集传递给 Lightning：
* 创建一个 DataLoader，它在后台迭代多个数据集。
* 在训练循环中，您可以将多个 DataLoader 作为字典或列表/元组传递，Lightning 将自动组合来自不同 DataLoader 的批次。
* 在验证和测试循环中，您可以选择返回多个 DataLoader，Lightning 将依次调用它们。

# 使用 LightningDataModule

你可以使用它的数据加载器钩子在你的 LightningDataModule 中设置多个 DataLoader，Lightning 将在后台使用正确的一个。

In [ ]:
class DataModule(LightningDataModule):

    ...

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset)

    def val_dataloader(self):
        return [torch.utils.data.DataLoader(self.val_dataset_1), torch.utils.data.DataLoader(self.val_dataset_2)]

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset)

# 使用 LightningModule 钩子

## 连接数据集

对于多个数据集的训练，您可以创建一个 dataloader 类来包装您的多个数据集（这当然也适用于测试和验证数据集）。

In [ ]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)


class LitModel(LightningModule):
    def train_dataloader(self):
        concat_dataset = ConcatDataset(datasets.ImageFolder(traindir_A), datasets.ImageFolder(traindir_B))

        loader = torch.utils.data.DataLoader(
            concat_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, pin_memory=True
        )
        return loader

    def val_dataloader(self):
        # SAME
        ...

    def test_dataloader(self):
        # SAME
        ...

## 返回多个 DataLoader

您可以在 LightningModule 中设置多个 DataLoader，Lightning 会负责批量组合。

有关更多详细信息，请查看 multiple_trainloader_mode

In [ ]:
class LitModel(LightningModule):
    def train_dataloader(self):

        loader_a = torch.utils.data.DataLoader(range(6), batch_size=4)
        loader_b = torch.utils.data.DataLoader(range(15), batch_size=5)

        # 将加载器作为字典传递。 这将创建这样的批次：
        # {'a': batch from loader_a, 'b': batch from loader_b}
        loaders = {"a": loader_a, "b": loader_b}

        # 或者：
        # 将加载器作为序列传递。 这将创建这样的批次：
        # [batch from loader_a, batch from loader_b]
        loaders = [loader_a, loader_b]

        return loaders

此外，Lightning 还支持嵌套列表和字典（或组合）。

In [ ]:
class LitModel(LightningModule):
    def train_dataloader(self):

        loader_a = torch.utils.data.DataLoader(range(8), batch_size=4)
        loader_b = torch.utils.data.DataLoader(range(16), batch_size=2)

        return {"a": loader_a, "b": loader_b}

    def training_step(self, batch, batch_idx):
        # 从每个 DataLoader 访问一个带有批处理的字典
        batch_a = batch["a"]
        batch_b = batch["b"]

In [ ]:
class LitModel(LightningModule):
    def train_dataloader(self):

        loader_a = torch.utils.data.DataLoader(range(8), batch_size=4)
        loader_b = torch.utils.data.DataLoader(range(16), batch_size=4)
        loader_c = torch.utils.data.DataLoader(range(32), batch_size=4)
        loader_c = torch.utils.data.DataLoader(range(64), batch_size=4)

        # 将加载器作为嵌套字典传递。 这将创建这样的批次：
        loaders = {"loaders_a_b": [loader_a, loader_b], "loaders_c_d": {"c": loader_c, "d": loader_d}}
        return loaders

    def training_step(self, batch, batch_idx):
        # access the data
        batch_a_b = batch["loaders_a_b"]
        batch_c_d = batch["loaders_c_d"]

        batch_a = batch_a_b[0]
        batch_b = batch_a_b[1]

        batch_c = batch_c_d["c"]
        batch_d = batch_c_d["d"]

# 多个验证/测试数据集

对于验证和测试 DataLoader，您可以传递单个 DataLoader 或它们的列表。 此可选命名参数可与上述任何用例结合使用。 您可以选择按顺序或同时传递批次，就像训练步骤一样。 验证和测试 DataLoaders 的默认模式是顺序的。

有关默认顺序选项的更多详细信息，请参阅以下内容：
* `val_dataloader()`
* `test_dataloader()`

In [ ]:
def val_dataloader(self):
    loader_1 = DataLoader()
    loader_2 = DataLoader()
    return [loader_1, loader_2]

要同时组合多个测试和验证 DataLoader 的批次，需要使用 CombinedLoader 包装 DataLoader。

In [ ]:
from pytorch_lightning.trainer.supporters import CombinedLoader


def val_dataloader(self):
    loader_1 = DataLoader()
    loader_2 = DataLoader()
    loaders = {"a": loader_a, "b": loader_b}
    combined_loaders = CombinedLoader(loaders, "max_size_cycle")
    return combined_loaders

# 使用额外的数据加载器进行测试

即使尚未在 LightningModule 实例中定义 test_dataloader() 方法，您也可以在测试集上运行推理。 例如，如果您的测试数据集在您的模型声明时不可用，就会出现这种情况。 只需将测试集传递给 test() 方法：

In [ ]:
# setup your data loader
test = DataLoader(...)

# test (pass in the loader)
trainer.test(test_dataloaders=test)

# 顺序数据

Lightning 内置了对处理顺序数据的支持。

## 打包序列作为输入

使用 PackedSequence 时，做两件事：
* 返回数据集中的填充张量或 DataLoader collate_fn 中的可变长度张量列表（示例显示列表实现）。
* 根据用例将序列打包到前向或训练和验证步骤中。

In [ ]:
# For use in DataLoader
def collate_fn(batch):
    x = [item[0] for item in batch]
    y = [item[1] for item in batch]
    return x, y


# In module
def training_step(self, batch, batch_nb):
    x = rnn.pack_sequence(batch[0], enforce_sorted=False)
    y = rnn.pack_sequence(batch[1], enforce_sorted=False)

# 时间截断断续传播 (TBPTT)

例如，在训练 RNN 时使用 Truncated Backpropagation Through Time 可以节省内存。

闪电可以通过这个标志自动处理TBPTT。

In [ ]:
from pytorch_lightning import LightningModule


class MyModel(LightningModule):
    def __init__(self):
        super().__init__()
        # 重要：这个属性激活了时间截断的反向传播
        # 将此值设置为 2 会将批次拆分为大小为 2 的序列
        self.truncated_bptt_steps = 2

    # 截断时间反向传播
    def training_step(self, batch, batch_idx, hiddens):
        # 必须更新训练步骤以接受 ``hiddens`` 参数
        # hiddens 是前一个被截断的反向传播步骤的隐藏
        out, hiddens = self.lstm(data, hiddens)
        return {"loss": ..., "hiddens": hiddens}

> 如果您需要修改批处理的拆分方式，请覆盖 tbptt_split_batch()。

# 可迭代数据集

Lightning 支持使用 IterableDatasets 以及地图样式的数据集。 IterableDatasets 在使用顺序数据时提供了一个更自然的选择。

> 使用 IterableDataset 时，您必须在初始化 Trainer 时将 val_check_interval 设置为 1.0（默认值）或 int（指定验证前要运行的训练批次数）。 这是因为 IterableDataset 没有 __len__ 并且当 val_check_interval 小于 1 时，Lightning 需要它来计算验证间隔。 同样，您可以将 limit_{mode}_batches 设置为浮点数或整数。 如果它设置为 0.0 或 0 它将设置 num_{mode}_batches 为 0，如果它是一个 int 它会将 num_{mode}_batches 设置为 limit_{mode}_batches，如果它设置为 1.0 它将运行 整个数据集，否则会抛出异常。 这里的模式可以是训练/验证/测试。

In [ ]:
# IterableDataset
class CustomDataset(IterableDataset):
    def __init__(self, data):
        self.data_source

    def __iter__(self):
        return iter(self.data_source)


# Setup DataLoader
def train_dataloader(self):
    seq_data = ["A", "long", "time", "ago", "in", "a", "galaxy", "far", "far", "away"]
    iterable_dataset = CustomDataset(seq_data)

    dataloader = DataLoader(dataset=iterable_dataset, batch_size=5)
    return dataloader

In [ ]:
# Set val_check_interval
trainer = Trainer(val_check_interval=100)

# Set limit_val_batches to 0.0 or 0
trainer = Trainer(limit_val_batches=0.0)

# Set limit_val_batches as an int
trainer = Trainer(limit_val_batches=100)